In [2]:
!wget https://mammographydata.s3.amazonaws.com/totaldata.zip

--2022-12-31 15:20:09--  https://mammographydata.s3.amazonaws.com/totaldata.zip
Resolving mammographydata.s3.amazonaws.com (mammographydata.s3.amazonaws.com)... 52.217.137.169, 52.216.244.180, 52.216.57.57, ...
Connecting to mammographydata.s3.amazonaws.com (mammographydata.s3.amazonaws.com)|52.217.137.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54411087734 (51G) [application/zip]
Saving to: ‘totaldata.zip’

totaldata.zip       100%[===================>]  50.67G  50.8MB/s    in 25m 0s  

2022-12-31 15:45:11 (34.6 MB/s) - ‘totaldata.zip’ saved [54411087734/54411087734]



In [3]:
!unzip -q totaldata.zip

In [11]:
%pip install pydicom

     |████████████████████████████████| 2.0 MB 25.5 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import torch
import pandas as pd
import pickle
import pydicom as dicom 
import numpy as np
import boto3 as boto

In [16]:
s3_resource = boto.resource('s3')
s3_client = boto.client('s3')

In [19]:
s3_client.list_buckets()

{'ResponseMetadata': {'RequestId': 'A46DKMC6A2WBB0XT',
  'HostId': 'sFzG6gs56mRunW+7mr7c3fw2H3s2Mjrapi8zpMaH8NyPSO/bJtTuFSTIRoKX6AIBQz5uVDvHaVg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sFzG6gs56mRunW+7mr7c3fw2H3s2Mjrapi8zpMaH8NyPSO/bJtTuFSTIRoKX6AIBQz5uVDvHaVg=',
   'x-amz-request-id': 'A46DKMC6A2WBB0XT',
   'date': 'Sat, 31 Dec 2022 22:00:11 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'mammographydata',
   'CreationDate': datetime.datetime(2022, 12, 27, 22, 23, 46, tzinfo=tzlocal())},
  {'Name': 'sagemaker-studio-076583500559-g8yws2424w',
   'CreationDate': datetime.datetime(2022, 12, 31, 15, 8, 6, tzinfo=tzlocal())},
  {'Name': 'sagemaker-us-east-1-076583500559',
   'CreationDate': datetime.datetime(2022, 12, 31, 15, 11, 14, tzinfo=tzlocal())}],
 'Owner': {'DisplayName': 'bls255',
  'ID': '233f5bf55406c96fd4037d43538bbfb7e3002791acf4ccc882bcab4982cfe336'}}

In [20]:
i =0
for patient in os.listdir('parent/images/0025a5dc99fd5c742026f0b2b030d3e9'):
    i += 1
print(f'There are {i} images')

There are 4 images


In [7]:
# parse csv summary file

df = pd.read_csv(f'parent/breast-level_annotations.csv')
df = df[(df['height'] == 3518) & (df['width'] == 2800)]
df = df[~df['study_id'].isin(['dbca9d28baa3207b3187c4d07dc81a80'])]

In [9]:
#  CREATE LABEL DICT of the form {'patient': {'L': score, 'R': score} }

patients = list(df['study_id'].unique())
left_breast_birads = df[((df['laterality'] == 'L') & (
    df['view_position'] == 'CC'))]['breast_birads']
right_breast_birads = df[((df['laterality'] == 'R') & (
    df['view_position'] == 'CC'))]['breast_birads']

left_breast_birads = list(left_breast_birads.map(lambda x: int(x[-1])))
right_breast_birads = list(right_breast_birads.map(lambda x: int(x[-1])))

label_dict = dict()

i = 0
for patient in patients:
    left_label = left_breast_birads[i]
    right_label = right_breast_birads[i]
    label_dict[patient] = {'L': left_label, 'R': right_label}
    i += 1

In [57]:
# Pickling dictionary and uploading it to s3 bucket
# Arugments go: (Name in notebook, s3 bucket, path in bucket/name)
pickle.dump(label_dict,open('label_dict.pt','wb'))
s3_client.upload_file('label_dict.pt', 'mammographydata', 'DataSet/label_dict.pt')
s3_client.upload_file('label_dict.pt', 'mammographydata', 'DataSet/Label_dict_folder/label_dict.pt')

In [63]:
def find_zeros(data):
    x , y = data.shape
    zero_t = torch.tensor(0)
    if x != 3500:
        diff = x - 3500
        i = 0
        j = 0
        k = 0
        while k != diff:
            #print(f'in loop {k} time on same data')
            if (torch.count_nonzero(data[:i,:]) == zero_t):
                i += 1
            if (torch.count_nonzero(data[x-j:x,:] == zero_t)):
                j += 1
            k += 1
            
        if i + j != diff:
            #print('There was an issue')
            i = 0
            j = diff
            
            
    return data[i:x-j,:]

In [ ]:
for patient in patients:
    curr_df = df[df['study_id'] == patient]
    lmlo_image_id = list(curr_df[((curr_df['laterality'] == 'L') & (
        curr_df['view_position'] == 'MLO'))]['image_id'])[0]
    lcc_image_id = list(curr_df[((curr_df['laterality'] == 'L') & (
        curr_df['view_position'] == 'CC'))]['image_id'])[0]
    rmlo_image_id = list(curr_df[((curr_df['laterality'] == 'R') & (
        curr_df['view_position'] == 'MLO'))]['image_id'])[0]
    rcc_image_id = list(curr_df[((curr_df['laterality'] == 'R') & (
        curr_df['view_position'] == 'CC'))]['image_id'])[0]

    lmlo_dicom = dicom.dcmread(f'parent/images/{patient}/{lmlo_image_id}.dicom')
    lcc_dicom = dicom.dcmread(f'parent/images/{patient}/{lcc_image_id}.dicom')
    rmlo_dicom = dicom.dcmread(f'parent/images/{patient}/{rmlo_image_id}.dicom')
    rcc_dicom = dicom.dcmread(f'parent/images/{patient}/{rcc_image_id}.dicom')
    

    lmlo_torch = find_zeros(torch.from_numpy(np.array(lmlo_dicom.pixel_array, dtype= np.float32)))
    lcc_torch = find_zeros(torch.from_numpy(np.array(lcc_dicom.pixel_array, dtype = np.float32)))
    rmlo_torch = find_zeros(torch.flip(torch.from_numpy(np.array(rmlo_dicom.pixel_array, dtype = np.float32)),(1,)))
    rcc_torch = find_zeros(torch.flip(torch.from_numpy(np.array(rcc_dicom.pixel_array, dtype= np.float32)),(1,)))
    
    

    pickle.dump(lmlo_torch, open(f'{patient}_LMLO.pt', "wb"))
    pickle.dump(lcc_torch, open(f'{patient}_LCC.pt', "wb"))
    pickle.dump(rmlo_torch, open(f'{patient}_RMLO_flipped.pt', "wb"))
    pickle.dump(rcc_torch, open(f'{patient}_RCC_flipped.pt', "wb"))
    
    s3_client.upload_file(f'{patient}_LMLO.pt', 'mammographydata', f'DataSet/processed/{patient}/LMLO.pt')
    s3_client.upload_file(f'{patient}_LCC.pt', 'mammographydata', f'DataSet/processed/{patient}/LCC.pt')
    s3_client.upload_file(f'{patient}_RMLO_flipped.pt', 'mammographydata', f'DataSet/processed/{patient}/RMLO_flipped.pt')
    s3_client.upload_file(f'{patient}_RCC_flipped.pt', 'mammographydata', f'DataSet/processed/{patient}/RCC_flipped.pt')

In [56]:
i = 0
for patient in os.listdir('parent/images'):
    for image in os.listdir(f'parent/images/{patient}'):
        ds = dicom.dcmread(f'parent/images/{patient}/{image}')
        data = np.array(ds.pixel_array, dtype = np.float32)
        data = torch.from_numpy(data)
        new_data = find_zeros(data)
        if new_data.shape != (3500,2800) and data.shape == (3518, 2800):
            print(f'Patient ID: {patient}')
            print(f'Image ID: {image}')
            print(data.shape)

There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue
There was an issue


KeyboardInterrupt: 